# Introducing Factory Method

In [4]:
import json
import xml.etree.cElementTree as et


class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist


class SongSerializer:
    def serialize(self, song, format):
        if format == "JSON":
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)
        elif format == "XML":
            song_info = et.Element("song", arrtib={song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')
        else:
            raise ValueError

In [7]:
song = Song('1', 'Water of Love', "Dire Straits")
serializer = SongSerializer()

In [8]:
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

serializer.serialize(song, 'XML')

In [18]:
serializer.serialize(song, "YAML")

ValueError: 

### Refactoring Code Into the Desired Interface

In [21]:
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            return self._serialize_to_json(song)
        elif format == 'XML':
            return self._serialize_to_xml(song)
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

### Basic Implementation of Factory Method

In [23]:
class SongSerializer:
    def serialize(self, song, format):
        serializer = self._get_serializer(format)
        return serializer(song)

    def _get_serializer(self, format):
        if format == 'JSON':
            return self._serialize_to_json
        elif format == 'XML':
            return self._serialize_to_xml
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

In [24]:
song = Song('1', 'Water of Love', "Dire Straits")
serializer = SongSerializer()

In [25]:
serializer.serialize(song, "JSON")

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [26]:
serializer.serialize(song, "XML")

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

In [27]:
serializer.serialize(song, "YAML")

ValueError: YAML

### An Object Serialization Example

In [25]:
import json
import xml.etree.ElementTree as et

class JsonSerializer:
    def __init__(self):
        self._current_object = None

    def start_object(self, object_name, object_id):
        self._current_object = {
            'id': object_id
        }

    def add_property(self, name, value):
        self._current_object[name] = value

    def to_str(self):
        return json.dumps(self._current_object)

In [68]:
class XmlSerializer:
    def __init__(self):
        self._element = None

    def start_object(self, object_name, object_id):
        self._element = et.Element(object_name, attrib={'id': object_id})

    def add_property(self, name, value):
        prop = et.SubElement(self._element, name)
        prop.text = value

    def to_str(self):
        return et.tostring(self._element, encoding='unicode')

In [69]:
# In songs.py

class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist

    def serialize(self, serializer):
        serializer.start_object('song', self.song_id)
        serializer.add_property('title', self.title)
        serializer.add_property('artist', self.artist)

### Factory Method as an Object Factory

In [70]:
class SerializerFactory:
    def get_serializer(self, format):
        if format == 'JSON':
            return JsonSerializer()
        elif format == 'XML':
            return XmlSerializer()
        else:
            raise ValueError(format)


factory = SerializerFactory()

In [71]:
class ObjectSerializer:
    def serialize(self, serializable, format):
        serializer = factory.get_serializer(format)
        serializable.serialize(serializer)
        return serializer.to_str()

In [72]:
song = Song('1', 'Water of Love', 'Dire Straits')

In [73]:
serializer = ObjectSerializer()

In [74]:
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [75]:
serializer.serialize(song, 'XML')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

In [76]:
serializer.serialize(song, 'YAML')

ValueError: YAML

### Supporting Additional Formats

In [83]:
class SerializeFactory:

    def __init__(self):
        self._creators = {}

    def register_format(self, format, creator):
        self._creators[format] = creator

    def get_serializer(self, format):
        creator = self._creators.get(format)
        if not creator:
            raise ValueError(format)
        return creator

In [84]:
factory = SerializeFactory()
factory.register_format('JSON', JsonSerializer)
factory.register_format('XML', XmlSerializer)

In [85]:
import yaml

class YamlSerializer(JsonSerializer):
    def to_str(self):
        return yaml.dump(self._current_object)

factory.register_format('YAML', YamlSerializer)

In [86]:
serializer = ObjectSerializer()

In [87]:
song = Song('1', 'Water of Love', 'Dire Straits')

In [88]:
serializer.serialize(song, 'JSON')

TypeError: JsonSerializer.start_object() missing 1 required positional argument: 'object_id'

In [89]:
song

# A General Purpose Object Factory

### Not All Objects Can Be Created Equal